In [2]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/st_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/st_test.csv')

train_X = train.drop(['grade'], axis=1)
train_y = train['grade']

test_X = test.drop(['grade'], axis=1)
test_y = test['grade']

In [3]:
# 파이프라인과 ColumnTransformer()를 사용하여 데이터 전처리 진행

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer # 열마다 다른 전처리기를 쓰고 싶을 때
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV

num_columns = train_X.select_dtypes('number').columns.tolist() # Index 객체를 일반적인 Python 리스트로 변환하기 위해서
cat_columns = train_X.select_dtypes('object').columns.tolist()

cat_preprocess = make_pipeline(
    OneHotEncoder(handle_unknown='ignore', sparse_output=False)
)

num_preprocess = make_pipeline(
    SimpleImputer(strategy= 'mean'),
    StandardScaler()
)

preprocess = ColumnTransformer(
    [("num", num_preprocess, num_columns),
     ("cat", cat_preprocess, cat_columns)]
)

#### 회귀 분석 알고리즘

1. K-Nearest Neighbors(KNN)

In [10]:
from sklearn.neighbors import KNeighborsRegressor

full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ('regressor', KNeighborsRegressor())
    ]
)

In [11]:
# knn 모형의 파라미터 명칭 확인

KNeighborsRegressor().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [16]:
# K = 5 ~ 10 까지 튜닝 파라미터로 설정
import numpy as np

knn_params = {'regressor__n_neighbors': np.arange(5, 10, 1)}

In [17]:
# GridSearchCV()를 활용하여 KNN 모형에 대한 파라미터 튜닝 진행 cv=3 3-fold

knn_search = GridSearchCV(estimator=full_pipe,
                          param_grid=knn_params,
                          cv = 3,
                          scoring= 'neg_mean_squared_error')

knn_search.fit(train_X, train_y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['famrel',
                                                                          'freetime',
                                                                          'goout',
                                                                          'Dalc',
                                                                          'Walc',
                                                                          'health',
                                                                          'absences']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['school',
                                                                          'sex',
                                                                          'paid'])])),
                                       ('regressor', KNeighborsRegressor())]),
             param_grid={'regressor__n_neighbors': array([5, 6, 7, 8, 9])},
             scoring='neg_mean_squared_error')

In [18]:
pd.DataFrame(knn_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008501,0.002928,0.004522,0.000815,5,{'regressor__n_neighbors': 5},-9.476279,-8.924235,-10.623529,-9.674681,0.707777,4
1,0.004140,0.000455,0.002738,0.000653,6,{'regressor__n_neighbors': 6},-9.290375,-8.851634,-10.933987,-9.691999,0.896298,5
2,0.003421,0.000262,0.002050,0.000268,7,{'regressor__n_neighbors': 7},-9.110109,-8.545738,-10.694118,-9.449988,0.909403,3
3,0.003063,0.000072,0.001937,0.000167,8,{'regressor__n_neighbors': 8},-9.122093,-8.554963,-10.609926,-9.428994,0.866549,2
4,0.003320,0.000205,0.001915,0.000175,9,{'regressor__n_neighbors': 9},-9.059432,-8.537400,-10.621932,-9.406254,0.885638,1


In [20]:
print('best 파라미터 조합:', knn_search.best_params_)
print('교착검증 MSE:', -knn_search.best_score_)

best 파라미터 조합: {'regressor__n_neighbors': 9}
교착검증 MSE: 9.406254468482771


In [22]:
# 모형 성능 평가
from sklearn.metrics import mean_squared_error
knn_pred = knn_search.predict(test_X)
print(mean_squared_error(test_y, knn_pred))

9.732772166105496


#### decision tree

- 알고리즘 특성상 변수 스케일에 영향을 받지 않음

In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ("regressor", DecisionTreeRegressor())
    ]
)

In [29]:
# 의사결정 나무 파라미터 명칭 확인
DecisionTreeRegressor().get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [30]:
from sklearn.model_selection import GridSearchCV

decisiontree_param = {'regressor__ccp_alpha': np.arange(0.01, 0.3, 0.05)}
decisiontree_search = GridSearchCV(estimator=full_pipe,
                                   param_grid = decisiontree_param,
                                   cv=5,
                                   scoring= 'neg_mean_squared_error')

decisiontree_search.fit(train_X, train_y)

print('best 파라미터 조합', decisiontree_search.best_params_)
print('교차검증 MSE', decisiontree_search.best_score_)

best 파라미터 조합 {'regressor__ccp_alpha': 0.26}
교차검증 MSE -9.403541096157653


In [32]:
# 모델 성능 평가

from sklearn.metrics import mean_squared_error

dt_pred = decisiontree_search.predict(test_X)
print('MSE:', mean_squared_error(test_y, dt_pred))

MSE: 10.23195890566565


#### 앙상블 학습
    - bagging
        - 의사결정나무는 일반적으로 분산이 높은 문제점이 있음 즉 데이터가 조금만 바뀌어도 예측값이 크게 흔들리게 됨
        - 여러 샘플의 평균을 구할 때, 해당 평균의 분산은 개별 샘플의 분산보다 작아짐 -> 이 방법을 활용해 모집단으로부터 많은 훈련데이터를 취하고 각 훈련데이터 별로 모델을 적합 시킨 후 예측값의 평균을 구하기
        - 훈련 데이터는 하나만 갖고 있기 때문에 부트스트랩 샘플링을 활용해 이를 보완
        - 부트스트랩: 원래 데이터에서 중복 허용으로 샘플링하여 새로운 데이터를 여러 개 만드는 기법입니다.
        - 부트스트랩된 여러 데이터셋에 각각 모델을 훈련시키고, 그 결과를 **평균(회귀) 또는 투표(분류)**로 종합(Aggregating) 하는 방식의 앙상블 기법

In [34]:
from sklearn.ensemble import BaggingRegressor
from sklearn.pipeline import Pipeline

full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ("regressor", BaggingRegressor())
    ]
)

In [37]:
# BaggingRegressor 파라미터 명칭 확인

BaggingRegressor().get_params()

{'base_estimator': 'deprecated',
 'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [41]:
Bagging_param = {'regressor__n_estimators':np.arange(10, 100, 20), 'regressor__random_state': [0]}
Bagging_search = GridSearchCV(estimator = full_pipe,
                              param_grid= Bagging_param,
                              cv=5,
                              scoring = 'neg_mean_squared_error')

Bagging_search.fit(train_X, train_y)

print('best 조합', Bagging_search.best_params_)
print('MSE', -Bagging_search.best_score_)

best 조합 {'regressor__n_estimators': 30, 'regressor__random_state': 0}
MSE 9.581004443482522


In [42]:
# 최종 테스트 데이터 적용

from sklearn.metrics import mean_squared_error
bag_pred = Bagging_search.predict(test_X)
print('MSE:', mean_squared_error(test_y, bag_pred))

MSE: 9.626060816498317


#### Random forest
- 전체 변수가 아닌 일부 변수만 고려
- 전체 변수를 활용할 경우 부트스트랩 샘플별로 생성된 의사결정나무는 대부분 비슷하여 다양성이 떨어지게 됨
- 반면 일부 변수만 고려할 경우 부트스트랩 샘플별로 서로 다른 의사결정나무 생성되므로, 분산 감소 효과가 커지게 됩니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

full_pipe = Pipeline(
    [
        ('preprocess', preprocess),
        ('regressor', RandomForestRegressor()) # 여기서 정한 regressor는 밑에 grid_search에 활용
    ]
)

In [48]:
RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
RandomForest_param = {'regressor__n_estimators': np.arange(100, 500, 100),
                      'regressor__max_features': ['sqrt'], 'regressor__random_state': [0]} # regressor_max_features: 사용할 일부 변수의 개수 n개의 변수 중 sqrt(n)개

RandomForest_search = GridSearchCV(estimator= full_pipe,
                                   param_grid= RandomForest_param,
                                   cv=5,
                                   scoring='neg_mean_squared_error')

RandomForest_search.fit(train_X, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['famrel',
                                                                          'freetime',
                                                                          'goout',
                                                                          'Dalc',
                                                                          'Walc',
                                                                          'health',
                                                                          'absences']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['school',
                                                                          'sex',
                                                                          'paid'])])),
                                       ('regressor', RandomForestRegressor())]),
             param_grid={'regressor__max_features': ['sqrt'],
                         'regressor__n_estimators': array([100, 200, 300, 400]),
                         'regressor__random_state': [0]},
             scoring='neg_mean_squared_error')

In [53]:
print('best 파라미터 조합',RandomForest_search.best_params_)
print(-RandomForest_search.best_score_)

best 파라미터 조합 {'regressor__max_features': 'sqrt', 'regressor__n_estimators': 400, 'regressor__random_state': 0}
9.418447711599324


In [54]:
from sklearn.metrics import mean_squared_error

rf_pred = RandomForest_search.predict(test_X)
print('MSE:', mean_squared_error(test_y, rf_pred))

MSE: 10.007126272089838


#### Gradient Boosting
- 단일 의사결정나무 모형은 일반적으로 분산이 높은 문제
- 그래디언트 부스팅은 여러 개의 모델을 결합한다는 점에서 배깅과 유사 / 모델의 잔차를 순차적으로 업데이트하여 모델의 성능 향상
- 단일 모형의 과적합 문제를 보완하기 위해 잔차를 업데이트 해나가며 천천히 학습한다는 장점 다만 과적합 문제 발생 가능성이 있으므로 적절한 파라미터

In [58]:
from sklearn.ensemble import GradientBoostingRegressor

full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ("regressor", GradientBoostingRegressor())
    ]
)

In [59]:
GradientBoostingRegressor().get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [62]:
GradientBoosting_param = {'regressor__learning_rate': np.arange(0.1, 0.3, 0.05), # 학습기의 강도 조절
                             'regressor__random_state': [0]}
GradientBoosting_search = GridSearchCV(estimator=full_pipe,
                                       param_grid= GradientBoosting_param,
                                       cv=5,
                                       scoring='neg_mean_squared_error')

GradientBoosting_search.fit(train_X, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['famrel',
                                                                          'freetime',
                                                                          'goout',
                                                                          'Dalc',
                                                                          'Walc',
                                                                          'health',
                                                                          'absences']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['school',
                                                                          'sex',
                                                                          'paid'])])),
                                       ('regressor',
                                        GradientBoostingRegressor())]),
             param_grid={'regressor__learning_rate': array([0.1 , 0.15, 0.2 , 0.25]),
                         'regressor__random_state': [0]},
             scoring='neg_mean_squared_error')

In [65]:
print('best parameter:', GradientBoosting_search.best_params_)
print('교차검증 MSE', -GradientBoosting_search.best_score_)

best parameter: {'regressor__learning_rate': 0.1, 'regressor__random_state': 0}
교차검증 MSE 10.788240022704802


In [67]:
from sklearn.metrics import mean_squared_error

gb_pred = GradientBoosting_search.predict(test_X)
print('테스트 MSE:', mean_squared_error(test_y, gb_pred))

테스트 MSE: 10.547041848465328


#### 고급 회귀 기법 (SVR, Support Vector Regression)
- Support Vector Machine은 보통 분류 문제에서 활용되지만 비슷한 아이디어를 회귀 문제에도 적용할 수 있음 
- 일반적으로 선형회귀는 squared loss를 최소화 하는 직선을 찾는다. 그러나 선형회귀는 이상치에 민감한 특성이 있다.
    - 이 때문에 Support Vector Regression은 &epsilon;-insensitive loss 사용
    - &epsilon;-insensitive loss는 실제값과 예측값 차이가 &epsilon;(앱실론) 이내일 경우 손실을 0으로 처리
    - 즉 &epsilon; 내에 있는 데이터 회귀직선을 피팅하는 데 영향을 주지 않는다.

In [68]:
from sklearn.svm import SVR
full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ("regressor", SVR())
    ]
)

In [69]:
SVR().get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [70]:
SVR_param = {'regressor__C': np.arange(1, 100, 20)}
SVR_search = GridSearchCV(estimator = full_pipe,
                          param_grid = SVR_param,
                          cv = 5,
                          scoring='neg_mean_squared_error')

SVR_search.fit(train_X, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['famrel',
                                                                          'freetime',
                                                                          'goout',
                                                                          'Dalc',
                                                                          'Walc',
                                                                          'health',
                                                                          'absences']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['school',
                                                                          'sex',
                                                                          'paid'])])),
                                       ('regressor', SVR())]),
             param_grid={'regressor__C': array([ 1, 21, 41, 61, 81])},
             scoring='neg_mean_squared_error')

In [ ]:
print('best 파라미터:', SVR_search.best_params_)
print('mse:', -SVR_search.best_score_)

best 파라미터: {'regressor__C': 1}
mse: 8.905507590431395


In [77]:
from sklearn.metrics import mean_squared_error

svr_pred = SVR_search.predict(test_X)
print('테스트 MSE:', mean_squared_error(test_y, svr_pred))

테스트 MSE: 10.141966042523617


#### 모범 답안 작성 예시

In [78]:
# 학생 성적 데이터

import pandas as pd
import numpy as np

train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/st_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/st_test.csv')

In [79]:
# 데이터 탐색 (결측치 혹은 특이치 확인)
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    256 non-null    object 
 1   sex       256 non-null    object 
 2   paid      256 non-null    object 
 3   famrel    256 non-null    int64  
 4   freetime  256 non-null    int64  
 5   goout     252 non-null    float64
 6   Dalc      256 non-null    int64  
 7   Walc      256 non-null    int64  
 8   health    256 non-null    int64  
 9   absences  256 non-null    int64  
 10  grade     256 non-null    int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 22.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    110 non-null    object 
 1   sex       110 non-null    object 
 2   paid      110 non-null    object 
 3   famrel    110 n

In [80]:
# 데이터 분할

train_X = train.drop(['grade'], axis=1)
train_y = train['grade']

test_X = test.drop(['grade'], axis=1)
test_y = test['grade']

from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)

(179, 10) (179,) (77, 10) (77,)


In [81]:
# 데이터 전처리
# 범주형 변수에 대해서 원-핫 인코딩 수행 / 결측치가 존재하는 컬럼에 대해 결측치 대치

# 범주형 변수와 수치형 변수 각 컬럼명 저장
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

# 원-핫 인코딩과 평균 대치법을 위한 메서드 불러오기
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

onehotencoder = OneHotEncoder(sparse_output=False, handle_unknown= 'ignore')
imputer = SimpleImputer(strategy='mean')

In [89]:
# 각각 데이터 전처리 실행 / 전처리 결과는 np.array 형태로 출력

# 수치형 전처리
train_X_numeric_imputed = imputer.fit_transform(train_X[num_columns])
valid_X_numeric_imputed = imputer.transform(valid_X[num_columns])
test_X_numeric_imputed = imputer.transform(test_X[num_columns])

# 범주형 원핫인코딩
train_X_categorical_encoded = onehotencoder.fit_transform(train_X[cat_columns])
valid_X_categorical_encoded = onehotencoder.transform(valid_X[cat_columns])
test_X_categorical_encoded = onehotencoder.transform(test_X[cat_columns])

# np.array 형태이기 때문에 np.concatenate 사용
train_X_preprocessed = np.concatenate([train_X_numeric_imputed, train_X_categorical_encoded], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_numeric_imputed, valid_X_categorical_encoded], axis=1)
test_X_preprocessed = np.concatenate([test_X_numeric_imputed, test_X_categorical_encoded], axis=1)

In [90]:
# 모델 적합

from sklearn.ensemble import RandomForestRegressor

rf= RandomForestRegressor(random_state=1)
rf.fit(train_X_preprocessed, train_y)

RandomForestRegressor(random_state=1)

In [91]:
# 모델 성능 확인

from sklearn.metrics import mean_squared_error

pred_val = rf.predict(valid_X_preprocessed)
print('valid MSE', mean_squared_error(valid_y, pred_val))
print('valid MSE', mean_squared_error(valid_y, pred_val, squared=False))

valid MSE 10.56478961038961
valid MSE 3.2503522286653195


In [93]:
# 테스트 데이터 예측

test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns=['pred'])
test_pred.to_csv('result.csv', index=False)

In [95]:
### 홀드 아웃 방법이 아닌 K-fold 교차검증 진행
# 기존 train, valid 합치기

train_X_full = np.concatenate([train_X_preprocessed, valid_X_preprocessed], axis=0) # 추가 데이터를 합치기 때문에 axis=0 행
train_y_full = np.concatenate([train_y, valid_y], axis=0)

In [96]:
RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# GridSearchCV() 하이퍼 파라미터 튜닝 / 하이퍼파라미터 값은 적당히

from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [10, 20, 30], # 한 결정트리가 얼마나 깊게 뻗어나갈 수 있는지 (클수록 과적합 위험)
              'min_samples_split': [2, 5, 10]} # 어떤 노드에 샘플이 이 값보다 작게 남으면 더 이상 분할하지 않음 (적을수록 과적합 위험)

rf = RandomForestRegressor(random_state=1)
rf_search = GridSearchCV(estimator=rf,
                         param_grid=param_grid,
                         cv= 3,
                         scoring='neg_mean_squared_error')

rf_search.fit(train_X_full, train_y_full)

print('교차검증 MSE', -rf_search.best_score_)
print('교차검증 RMSE', np.sqrt(-rf_search.best_score_))

교차검증 MSE 8.73209066811596
교차검증 RMSE 2.9550111113354482


In [101]:
# 테스트 데이터 최종 예측 후 저장

test_pred2 = rf_search.predict(test_X_preprocessed)
test_pred2 = pd.DataFrame(test_pred2, columns=['pred'])

test_pred2.to_csv('result.csv', index=False)